In [125]:
import numpy
import pandas as pd
from colormath import color_objects, color_diff
import math

def patch_asscalar(a):
    return a.item()
setattr(numpy, "asscalar", patch_asscalar)

color_df = pd.read_csv("all_colors.csv")

In [126]:
lab_string = list(color_df["L*a*b Value"])
lab_strings = [color_string.split(", ") for color_string in lab_string]

lab_floats = [[float(x) for x in row] for row in lab_strings]

lab_colors = [color_objects.LabColor(lab_l=lab_color[0], lab_a=lab_color[1], lab_b=lab_color[2]) for lab_color in lab_floats]

In [127]:
distance_matrix = {}
for i, color1 in enumerate(lab_colors):
    for j, color2 in enumerate(lab_colors):
        if i != j:
            distance_matrix[(i, j)] = round(color_diff.delta_e_cie2000(color1=color1, color2=color2) * 10)
            #distance_matrix[(i, j)] = round(math.dist((color1.lab_l, color1.lab_a, color1.lab_b),(color2.lab_l, color2.lab_a, color2.lab_b))*10)


In [128]:
selections_df = pd.read_csv("Team_Color_Selections.csv")
team_choices = {}
selections_df.head()
for idx, row in selections_df.iterrows():
    selected_color_A = row["Option_A"]
    selected_color_B = row["Option_B"]
    selected_color_C = row["Option_C"]

    index_A = color_df.index[color_df["Color Name"] == selected_color_A][0]
    index_B = color_df.index[color_df["Color Name"] == selected_color_B][0]
    index_C = color_df.index[color_df["Color Name"] == selected_color_C][0]

    team_choices[idx] = set([index_A, index_B, index_C])

len(team_choices)

17

In [129]:
from ortools.linear_solver import pywraplp

solver = pywraplp.Solver.CreateSolver('SCIP')

num_colors = 103
num_colors_to_select = 17

x = {} #x[c] = 1 if color is selected 
for c in range(num_colors):
    x[c] = solver.BoolVar(f'select_color_{c}')

y = {} #y[t][c] if team t is assigned color c
for t in team_choices:
    y[t] = {}
    for c in range(num_colors):
        y[t][c] = solver.BoolVar(f'assign_team_{t}_color_{c}')

min_dist = solver.NumVar(0, float('inf'), 'min_distance')

# Constraints
#1. Select only 17 colors (17 teams)
solver.Add(sum(x[c] for c in range(num_colors)) == num_colors_to_select)
#2. Each team gets only one color
for t in team_choices:
    solver.Add(sum(y[t][c] for c in range(num_colors)) == 1)
#3. Teams only get colors they choose 
for t in team_choices:
    for c in range(num_colors):
        if c not in team_choices[t]:
            solver.Add(y[t][c] == 0)
#4 colors can only be assigned if selected
for t in team_choices:
    for c in range(num_colors):
        solver.Add(y[t][c] <= x[c])
#5 Each selected color is assigned to exactly one team
for c in range(num_colors):
    solver.Add(sum(y[t][c] for t in team_choices) == x[c])

# M = max(distance_matrix.values()) * 2
# for i in range(num_colors):
#     for j in range(i + 1, num_colors):
#         if (i, j) in distance_matrix:
#             dist = distance_matrix[(i, j)]
#         elif (j, i) in distance_matrix:
#             dist = distance_matrix[(j, i)]
#         else:
#             continue
#         solver.Add(min_dist <= dist + M * (2 - x[i] - x[j]))
# solver.Maximize(min_dist)

objective_terms = []
for i in range(num_colors):
    for j in range(i + 1, num_colors):
        if (i, j) in distance_matrix:
            dist = distance_matrix[(i, j)]
        elif (j, i) in distance_matrix:
            dist = distance_matrix[(j, i)]
        else:
            continue
        
        # Add distance * (x[i] * x[j]) to objective
        # Since x[i] and x[j] are binary, x[i] * x[j] = 1 iff both selected
        pair_var = solver.BoolVar(f'pair_{i}_{j}')
        solver.Add(pair_var <= x[i])
        solver.Add(pair_var <= x[j])
        solver.Add(pair_var >= x[i] + x[j] - 1)
        
        objective_terms.append(dist * pair_var)
solver.Maximize(sum(objective_terms))


status = solver.Solve()

In [130]:
if status == pywraplp.Solver.OPTIMAL:
    # Extract solution
    selected_colors = [c for c in range(num_colors) if x[c].solution_value() > 0.5]
    team_assignments = {}
    for t in team_choices:
        for c in range(num_colors):
            if y[t][c].solution_value() > 0.5:
                team_assignments[t] = c
                break
    
    # Calculate actual average distance
    total_dist = 0
    count = 0
    for i in selected_colors:
        for j in selected_colors:
            if i < j:
                if (i, j) in distance_matrix:
                    total_dist += distance_matrix[(i, j)]
                elif (j, i) in distance_matrix:
                    total_dist += distance_matrix[(j, i)]
                count += 1

    avg_dist = total_dist / count if count > 0 else 0
    print(selected_colors)
    print(team_assignments)
else:
    print("Error")
    raise Exception(f"Solver failed with status: {status}")

[6, 7, 12, 16, 17, 18, 19, 20, 24, 25, 33, 47, 52, 53, 76, 85, 88]
{0: 33, 1: 18, 2: 76, 3: 88, 4: 52, 5: 6, 6: 16, 7: 12, 8: 19, 9: 85, 10: 53, 11: 25, 12: 24, 13: 20, 14: 17, 15: 47, 16: 7}


In [131]:
team_assignments.items()

dict_items([(0, 33), (1, 18), (2, 76), (3, 88), (4, 52), (5, 6), (6, 16), (7, 12), (8, 19), (9, 85), (10, 53), (11, 25), (12, 24), (13, 20), (14, 17), (15, 47), (16, 7)])

In [132]:
def hex_to_rgb(hex_color):
    hex_color = hex_color.lstrip('#')
    return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))

def print_color_block_hex(hex_color, text="    "):
    r, g, b = hex_to_rgb(hex_color)
    print(f"\033[48;2;{r};{g};{b}m{text}\033[0m", end="")

for team, color in team_assignments.items():
    print_color_block_hex(color_df.iloc[color]['Hex'])
    print(f" {selections_df.iloc[team]['Team']}: {color_df.iloc[color]['Color Name']}")

print(f"Average Distance: {avg_dist/10}")

     Load Bearing Balls: Orange
     Piper Throw: Forest
     Holding Space Serving Face: Heather Midnight Navy
     Throwing Backshots: Heather Prism Mint
     PokeHoes: White
     America's Next Top: Berry
     K-Pop Dodgeball Hunters: Deep Teal
     Bad Girls Club: Charity Pink
     Divine Daddies: Gold
     No Dodge Reflex: Heather Prism Dusty Blue
     Duck Me, Daddy: Yellow
     Blastphemous Boys: Maroon
     Skibidi Slingers: Maize Yellow
     Mighty Moaning Power Bottoms: Kelly
     Valkyrie: Evergreen
     Holey Spirits: Team Purple
     Dodging Feelings: Black
Average Distance: 47.159558823529416


- Euclidean Distance by Max Avg: 79.35 Euclid Units
- Euclidean Distance by Maximin: 74.67 Eucld Units
- CIE 2000 by Max Avg: 47.16 CIE Units
- CIE 2000 by Maximin: 44.42 CIE Units